In [30]:
T1 = 25 #Wet_Solid inlet temperature  C
Ta = 30 #Mixed air inlet temperature C
Ca = 1.005 #specifc heat capacity of air kJ/kg.K
Cw = 4.180 #specifc heat capacity of water kJ/kg.K
Cs = 0.840 # specifc heat capacity of dry solid kJ/kg.K  check again
Cv = 2 # specifc heat capacity of vapor kJ/kg.K  
RH_in = 60 # relative humdity value of ventilation air = 60%,
Dt = 1 # required drying time hour
Fbp = 0.3 
Yba = 0.157
Y1 = 0.016 # inl
Y2 = (Fbp * Y1) + (1-Fbp)*Yba
Ms = 2000  #kg/h
Ms = 2000/3600 #kg/s
MCi = 45.6  #%
MCf = 15
X1 = MCi/(100-MCi) # inital moisture content in dry basis
X2 = MCf/(100-MCf) # inital moisture content in dry basis
Mds = Ms * ((100-MCi)/100)  #kg/s

ma = - Mds * (X1 - X2)/(Dt *(Y1 - Y2))   #dry air mass rate kg/sec
Tm = 60 #control final temperature C


CONV = (MCi - MCf)/(100-MCf)
Mss = Ms * CONV 
Cp = ((MCi* Cw) + ((100-MCi)* Cs))/ 100
Lv = 2300  #kJ/kg
Ql = Mss * Lv # latent heat energy of vaporisation kJ
#Qsens = Ms * Cp * (100-20) # energy required to heat the wet sample to the water boiling point
#Energy_required = Ql + Qsens


Qloss = 0 # assuming the dryer is thermally perfect 
Hai = ((Ca + (Cv * Y1))* Ta) +(Lv*Y1) # inlet hot air enthalpy kJ/kg
Haf = ((Ca + (Cv * Y2))* Tm) + (Lv*Y2) # outlet moist air enthalpy kJ/kg
Hsi = (Cs + (Cw * X1)) * T1 # inlet sample enthalpy 
Hsf = (Cs + (Cw * X2)) * Tm  # outlet sample enthalpy 
Qmw = (ma * (Haf - Hai)) + (Mds *(Hsf - Hai)) + Qloss  # microwave thermal energy [kW]
#Qcv = Ma * Cpa (Tai - Tf) # heat from the hot mixed air flow cross flowing through the dryer system 
Ql = Mss * Lv # latent heat energy of vaporisation 
Qsen = Ms * Cp * (Tm - T1)  # sensible heat required to heat the sample
Qmw
Qt = Ql + Qsen + Qloss #[kW]
Qt
print(Qmw,Qt)

555.1236025667007 505.9480000000001


In [48]:
import pandas as pd 
import math 

#Constant parameters 
T1 = 20 #Wet_Solid inlet temperature  C
Ta = 80 #Mixed air inlet temperature C
Ta1 = 30 # assumed initial air temperature C
Ca = 1.005 #specifc heat capacity of air kJ/kg.K
Cw = 4.180 #specifc heat capacity of water kJ/kg.K
Cs = 0.750 # specifc heat capacity of dry solid kJ/kg.K  
Cv = 2.0 # specifc heat capacity of vapor kJ/kg.K  
RH_in = 60 # relative humdity value of ventilation air = 60%,
Dt = 1 # required drying time hour
Fbp = 0.3 #bypass factor
Yba = 0.157 
Y1 = 0.016 # inlet absolute humidity (kg water/kg dry air)

#input variable parameters
Ms = 20  # wet sample weight kg 
MCi = 50  # initial moisture content wet basis
MCf = 0   # final moisture content wet basis %
X1 = MCi/(100-MCi) # inital moisture content in dry basis (kg water/kg dry solid)
X2 = MCf/(100-MCf) # inital moisture content in dry basis (kg water/kg dry solid)
Tm = 60 #control dryer final temperature C
P = 1.00 #dryer pressure (bar)

#calculated parameters 
Mds = Ms * ((100-MCi)/100)  #dry sample weight (kg)
Y2 = (Fbp * Y1) + (1-Fbp)*Yba # final absolute humidity (kg water/kg dry air)
Dt = Dt* 3600 # drying time in seconds (s)
ma = - Mds * (X1 - X2)/(Dt *(Y1 - Y2))   #dry air mass flow rate kg/sec
mda = ma/ ((1-Y1)/1) # drying air mass flow rate kg/sec
CONV = (MCi - MCf)/(100-MCf)  # dryer system conversion rate 
Msw = Ms * CONV #the amount of water removed (kg)
Cp = ((MCi* Cw) + ((100-MCi)* Cs))/ 100  # specific capacity of the sample

# latent energy of vaporisation based on the dryer pressure
df = pd.read_excel("steam_table.xlsx")
#Lv = 2300  #latent heat of vaporisation (kJ/kg) 
Lv = df.loc[df['pressure (bar)'] == P, 'Latent heat of vaporization(kJ/kg)'].item()

Qloss = 0 # assuming the dryer is thermally perfect 
Qeva = Msw * Lv/Dt   #latent heat energy of vaporisation (kW)
ha1 = ((Ca + (Cv * Y1))* Ta)
ha2 = ((Ca + (Cv * Y2))* Tm)
ha0 = ((Ca + (Cv * Y2))* Ta1)
Qg = ma * (ha1 - ha2) # energy from the dry gas into the dryer (kW)
dhs = Cs * (Tm - T1) # enthalpy change of the dry sample (kJ/kg)
Qs = Mds * dhs/Dt #  energy absorbed by the dry sample (kW)
Qmw = Qs - (Qeva + Qloss + Qg)  # Microwave thermal power required (kW)
Qha = mda * (ha0 - ha1) # energy from the drying air during drying time (kW)

print(Qmw, Qha)

-6.439032534053811 -1.313598917819641
